This is My first EDA Notebook , So give a comment if i have done any mistakes in this EDA. 
Your client is an Insurance company that has provided Health Insurance to its customers now they need your help in building a model to predict whether the policyholders (customers) from past year will also be interested in Vehicle Insurance provided by the company.

## *Hypothesis*
* Are customers with **low annual premium**,   **Interested** in insurance ?  
* Are **vintage customers** intersted in insurance?
* Are **Licensed customers** **interested** in insurance
* Are customers **interested** in insurance  **when vehicle has a damage**
* Are Customers **interested** in insurance when **vehicle age <1 year(new bike)**
* Are **previously insured** customers, **not interested** in insurance


# Exploratory  Data Analysis

In [ ]:
# load the libraries
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder
from sklearn.model_selection import StratifiedKFold

In [ ]:
# read data
rawdf = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/train.csv")
test  = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/test.csv")

In [ ]:
#see the head of data
rawdf.head()

# Varriable Identification and TypeCasting

In [ ]:
# get the data types of all features
rawdf.dtypes

### Integer Data type

In [ ]:
#get integer data types
rawdf.dtypes[rawdf.dtypes=="int64"]

In [ ]:
# Driving_License, Previously_Insured,Response is a categorical varriable so we convert it into category
rawdf["Driving_License"] = rawdf["Driving_License"].astype("category") 
rawdf["Previously_Insured"] = rawdf["Previously_Insured"].astype("category") 
rawdf["Driving_License"] = rawdf["Driving_License"].astype("category") 
rawdf["Response"] = rawdf["Response"].astype("category")

### Float data type

In [ ]:
# get float data types 
rawdf.dtypes[rawdf.dtypes == "float64"]

In [ ]:
#Region code , Policy Sales Channel are categorical varriables
rawdf["Region_Code"] = rawdf["Region_Code"].astype("category")
rawdf["Policy_Sales_Channel"] = rawdf["Policy_Sales_Channel"].astype("category")

### Object data type

In [ ]:
#vehicle age and vehicle damage is object varriable
rawdf.dtypes[rawdf.dtypes=="object"]

In [ ]:
# gender, vehicle age , vehicle damage are categorical varriables
rawdf["Gender"] = rawdf["Gender"].astype("category")
rawdf["Vehicle_Age"] = rawdf["Vehicle_Age"].astype("category")
rawdf["Vehicle_Damage"] = rawdf["Vehicle_Damage"].astype("category")

In [ ]:
# check all features data types after conversion
rawdf.dtypes

# Univariate Analysis

In [ ]:
# describe of all numeric values
rawdf.describe()

### Univariate Analysis Integer

In [ ]:
def UVA_numeric(data, var_group):
  ''' 
  Univariate_Analysis_numeric
  takes a group of variables (INTEGER and FLOAT) and plot/print all the descriptives and properties along with KDE.

  Runs a loop: calculate all the descriptives of i(th) variable and plot/print it
  '''

  size = len(var_group)
  plt.figure(figsize = (7*size,3), dpi = 100)
  
  #looping for each variable
  for j,i in enumerate(var_group):
    
    # calculating descriptives of variable
    mini = data[i].min()
    maxi = data[i].max()
    ran = data[i].max()-data[i].min()
    mean = data[i].mean()
    median = data[i].median()
    st_dev = data[i].std()
    skew = data[i].skew()
    kurt = data[i].kurtosis()

    # calculating points of standard deviation
    points = mean-st_dev, mean+st_dev

    #Plotting the variable with every information
    plt.subplot(1,size,j+1)
    sns.kdeplot(data[i], shade=True)
    sns.lineplot(points, [0,0], color = 'black', label = "std_dev")
    sns.scatterplot([mini,maxi], [0,0], color = 'orange', label = "min/max")
    sns.scatterplot([mean], [0], color = 'red', label = "mean")
    sns.scatterplot([median], [0], color = 'blue', label = "median")
    plt.xlabel('{}'.format(i), fontsize = 20)
    plt.ylabel('density')
    plt.title('std_dev = {}; kurtosis = {};\nskew = {}; range = {}\nmean = {}; median = {}'.format((round(points[0],2),round(points[1],2)),
                                                                                                   round(kurt,2),
                                                                                                   round(skew,2),
                                                                                                   (round(mini,2),round(maxi,2),round(ran,2)),
                                                                                                   round(mean,2),
                                                                                                   round(median,2)))

In [ ]:
# get numeric varriables
rawdf.select_dtypes(include=['int64','float64','Int64']).dtypes

In [ ]:
#Segregating varriables into groups
customer_details = ["Age","Vintage"]

In [ ]:
UVA_numeric(rawdf,customer_details)

In [ ]:
UVA_numeric(rawdf,["Annual_Premium"])

#### summary
* Most of the customers Age between 20 to 30 and 40 to 50 some peak there.
* vintage is normaly distributed. Average vintage value is 150
* Annual Premium is Highly skewed and also more kurtosis value so it has a extreme outliers 

### Univariate Analysis - Category

In [ ]:
def UVA_category(data, var_group):

  '''
  Univariate_Analysis_categorical
  takes a group of variables (category) and plot/print all the value_counts and barplot.
  '''
  # setting figure_size
  size = len(var_group)
  plt.figure(figsize = (7*size,5), dpi = 100)

  # for every variable
  for j,i in enumerate(var_group):
    n_uni = data[i].nunique()
    if n_uni > 20:
        norm_count1 = data[i].value_counts(normalize = True)
        norm_count = norm_count1.sort_values().tail(20) 
    else:
        norm_count = data[i].value_counts(normalize = True)
    

  #Plotting the variable with every information
    plt.subplot(1,size,j+1)
    sns.barplot(norm_count, norm_count.index , order = norm_count.index)
    plt.xlabel('fraction/percent', fontsize = 20)
    plt.ylabel('{}'.format(i), fontsize = 20)
    plt.title('n_uniques = {} \n value counts \n {};'.format(n_uni,norm_count))

In [ ]:
rawdf.select_dtypes(include=["category"]).dtypes

In [ ]:
# top 53 region code are taken
UVA_category(rawdf,["Gender","Driving_License","Region_Code"])

### summary
* Male customers higher than Female
* 99 % customers has a License
* Region Code 28 has a high number of customers
* Region Code 8, 46, 41 has a second high number of customers

In [ ]:
UVA_category(rawdf,["Vehicle_Age","Vehicle_Damage"])

### summary
* 95% Vechicle's age "within 2 years"
* Vehicle Damage is equally splitted

In [ ]:
UVA_category(rawdf,["Policy_Sales_Channel","Previously_Insured"])

### summary
* Most used ploicy sales channel code(152,26,124,160,156,122,157,154)
* Previously Insured people was less

In [ ]:
UVA_category(rawdf,["Response"])

#### summary
* Most of the customers not interested in insurance
* 99% customers are licensed  and 87% customers are not interested in insurance. so hyposthesis " **licensed customer** **interested** in insurance" is false   
* Imbalanced target Varriable 

# Univariate - Missing Values and Outlier Analysis

In [ ]:
rawdf.isnull().sum()

In [ ]:
# custom function for easy outlier analysis

def UVA_outlier(data, var_group, include_outlier = True):
  '''
  Univariate_Analysis_outlier:
  takes a group of variables (INTEGER and FLOAT) and plot/print boplot and descriptives\n
  Runs a loop: calculate all the descriptives of i(th) variable and plot/print it \n\n

  data : dataframe from which to plot from\n
  var_group : {list} type Group of Continuous variables\n
  include_outlier : {bool} whether to include outliers or not, default = True\n
  '''

  size = len(var_group)
  plt.figure(figsize = (7*size,4), dpi = 100)
  
  #looping for each variable
  for j,i in enumerate(var_group):
    
    # calculating descriptives of variable
    quant25 = data[i].quantile(0.25)
    quant75 = data[i].quantile(0.75)
    IQR = quant75 - quant25
    med = data[i].median()
    whis_low = med-(1.5*IQR)
    whis_high = med+(1.5*IQR)

    # Calculating Number of Outliers
    outlier_high = len(data[i][data[i]>whis_high])
    outlier_low = len(data[i][data[i]<whis_low])

    if include_outlier == True:
      #Plotting the variable with every information
      plt.subplot(1,size,j+1)
      sns.boxplot(data[i], orient="v")
      plt.ylabel('{}'.format(i))
      plt.title('With Outliers\nIQR = {}; Median = {} \n 2nd,3rd  quartile = {};\n Outlier (low/high) = {} \n'.format(
                                                                                                   round(IQR,2),
                                                                                                   round(med,2),
                                                                                                   (round(quant25,2),round(quant75,2)),
                                                                                                   (outlier_low,outlier_high)
                                                                                                   ))
      
    else:
      # replacing outliers with max/min whisker
      data2 = data[var_group][:]
      data2[i][data2[i]>whis_high] = whis_high+1
      data2[i][data2[i]<whis_low] = whis_low-1
      
      # plotting without outliers
      plt.subplot(1,size,j+1)
      sns.boxplot(data2[i], orient="v")
      plt.ylabel('{}'.format(i))
      plt.title('Without Outliers\nIQR = {}; Median = {} \n 2nd,3rd  quartile = {};\n Outlier (low/high) = {} \n'.format(
                                                                                                   round(IQR,2),
                                                                                                   round(med,2),
                                                                                                   (round(quant25,2),round(quant75,2)),
                                                                                                   (outlier_low,outlier_high)
                                                                                                   ))

In [ ]:
UVA_outlier(rawdf,["Annual_Premium","Age","Vintage"] )

### summary
* Annual premium has a extreme outlier

# Bivariate Analysis

In [ ]:
numerical_data = rawdf.select_dtypes(include=["int64","Int64","float64"])
numerical_data.corr()

In [ ]:
# plotting heatmap usill all methods for all transaction variables
plt.figure(figsize=(36,6), dpi=140)
for j,i in enumerate(['pearson']):
  plt.subplot(1,3,j+1)
  correlation = numerical_data.dropna().corr(method=i)
  sns.heatmap(correlation, linewidth = 2)
  plt.title(i, fontsize=18)

### summary

* Features dont have a strong correlation



### ScatterPlot

In [ ]:
# scatter plot for all numerical varriables
plt.figure(dpi=140)
sns.pairplot(numerical_data)

## Bivariate Analysis: Continuous-Categorical variables

In [ ]:
def TwoSampZ(X1, X2, sigma1, sigma2, N1, N2):
  '''
  takes mean, standard deviation, and number of observations and returns p-value calculated for 2-sampled Z-Test
  '''
  from numpy import sqrt, abs, round
  from scipy.stats import norm
  ovr_sigma = sqrt(sigma1**2/N1 + sigma2**2/N2)
  z = (X1 - X2)/ovr_sigma
  pval = 2*(1 - norm.cdf(abs(z)))
  return pval

In [ ]:
def TwoSampT(X1, X2, sd1, sd2, n1, n2):
  '''
  takes mean, standard deviation, and number of observations and returns p-value calculated for 2-sample T-Test
  '''
  from numpy import sqrt, abs, round
  from scipy.stats import t as t_dist
  ovr_sd = sqrt(sd1**2/n1 + sd2**2/n2)
  t = (X1 - X2)/ovr_sd
  df = n1+n2-2
  pval = 2*(1 - t_dist.cdf(abs(t),df))
  return pval

In [ ]:
def Bivariate_cont_cat(data, cont, cat, category):
  #creating 2 samples
  x1 = data[cont][data[cat]==category][:]
  x2 = data[cont][~(data[cat]==category)][:]
  
  #calculating descriptives
  n1, n2 = x1.shape[0], x2.shape[0]
  m1, m2 = x1.mean(), x2.mean()
  std1, std2 = x1.std(), x2.mean()
  
  #calculating p-values
  t_p_val = TwoSampT(m1, m2, std1, std2, n1, n2)
  z_p_val = TwoSampZ(m1, m2, std1, std2, n1, n2)

  #table
  table = pd.pivot_table(data=data, values=cont, columns=cat, aggfunc = np.mean)

  #plotting
  plt.figure(figsize = (15,6), dpi=140)
  
  #barplot
  plt.subplot(1,2,1)
  sns.barplot([str(category),'not {}'.format(category)], [m1, m2])
  plt.ylabel('mean {}'.format(cont))
  plt.xlabel(cat)
  plt.title('t-test p-value = {} \n z-test p-value = {}\n {}'.format(t_p_val,
                                                                z_p_val,
                                                                table))

  # boxplot
  plt.subplot(1,2,2)
  sns.boxplot(x=cat, y=cont, data=data)
  plt.title('categorical boxplot')
  

### Are **vintage customers** interested in insurance?

In [ ]:
Bivariate_cont_cat(rawdf, 'Vintage', 'Response', 1)

### summary
* No significant difference in vintage customers . so reject this hypothesis 
* both interested and no interested customers mean vintage is approxiamately same. no significant difference is there.so vintage has no impact on customer interest
* No outliers 

### Are aged customers interested in insurance?

In [ ]:
Bivariate_cont_cat(rawdf, 'Age', 'Response', 1)

 ### Inference
 * Interested Customers age between 35 to 52.
 * Uninterested Customers age between 25 to 48.
 * p-value of z and t tests  is 0 , so significant difference there. It indicates age makes a impact on customer interest
 
 

### Are customers with **low annual premium**,   **Interested** in insurance ?

In [ ]:
Bivariate_cont_cat(rawdf, 'Annual_Premium', 'Response', 1)

### Inference
* Annual premium mean for interested customer is 30419
* Annual premium mean for not interested customer is 31604
* Annual Premium makes impact on customer interest


## Bivariate Analysis(categorical categorical)

In [ ]:
def BVA_categorical_plot(data, tar, cat):
  '''
  take data and two categorical variables,
  calculates the chi2 significance between the two variables 
  and prints the result with countplot & CrossTab
  '''
  #isolating the variables
  data = data[[cat,tar]][:]

  #forming a crosstab
  table = pd.crosstab(data[tar],data[cat],)
  f_obs = np.array([table.iloc[0][:].values,
                    table.iloc[1][:].values])

  #performing chi2 test
  from scipy.stats import chi2_contingency
  chi, p, dof, expected = chi2_contingency(f_obs)
  
  #checking whether results are significant
  if p<0.05:
    sig = True
  else:
    sig = False

  #plotting grouped plot
  sns.countplot(x=cat, hue=tar, data=data)
  plt.title("p-value = {}\n difference significant? = {}\n".format(round(p,8),sig))

  #plotting percent stacked bar plot
#   sns.catplot(ax, kind='stacked')
  ax1 = data.groupby(cat)[tar].value_counts(normalize=True).unstack()
  ax1.plot(kind='bar', stacked='True',title=str(ax1))
  int_level = data[cat].value_counts()

### Are licensed customer has a more interest?

In [ ]:
BVA_categorical_plot(rawdf, "Response","Driving_License")

### Inference
* significant differnece is there.But 99%(univariate analysis) customers has a license, In that only 1 % customers are interested others are not interested  

In [ ]:
BVA_categorical_plot(rawdf, "Response","Previously_Insured")

### Inference
* Previously Insured customers not interested in insurance

In [ ]:
BVA_categorical_plot(rawdf, "Response","Vehicle_Damage")

### Inference
* Vehicle damaged customers has a interest in insurance 

In [ ]:
BVA_categorical_plot(rawdf, "Response","Vehicle_Age")

### Inference
* Overall vehicle Age 1-2 year category , interested customer's count is high.but
* In >2 year Age category, Interested customers percentage is high when compared to other category   

In [ ]:
BVA_categorical_plot(rawdf, "Response","Gender")

* Male customers has more interest in insurance. 

### Bivariate Analysis - Policy_Sales_Channel vs Response

In [ ]:
# Sales Channel wise Total Customers Count
psc_total = rawdf.groupby("Policy_Sales_Channel")["Response"].count()\
                          .reset_index(name='count') \
                             .sort_values(['count'], ascending=False)
psc_total

### summary
* In univarite Analysis, Already we have seen Policy sales channel 152 has a more entry

In [ ]:
# filter interested customers count 
FormattedData = rawdf[rawdf["Response"]==1].groupby("Policy_Sales_Channel")["Response"].count()\
                          .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) 

FormattedData["total_count"] = psc_total["count"]
FormattedData["success_rate"] = (FormattedData["count"]/FormattedData["total_count"])*100 
FormattedData[FormattedData["total_count"]>500].sort_values("success_rate",ascending=False).head(50)

### summary
* policy sales channel 155 has high success rate(32%) when compared to other channels.
* policy sales channel 26 has a more intersted customers. but success rate is 19%


### Bivariate Analysis(Region_code vs Response)

In [ ]:
# Sales Channel wise Total Customers Count
psc_total = rawdf.groupby("Region_Code")["Response"].count()\
                          .reset_index(name='count') \
                             .sort_values(['count'], ascending=False)
psc_total.head(25)

In [ ]:
# filter interested customers count 
FormattedData = rawdf[rawdf["Response"]==1].groupby("Region_Code")["Response"].count()\
                          .reset_index(name='count') \
                             .sort_values(['count'], ascending=False) 

FormattedData["total_count"] = psc_total["count"]
FormattedData["success_rate"] = (FormattedData["count"]/FormattedData["total_count"])*100 
FormattedData[FormattedData["total_count"]>500].sort_values("success_rate",ascending=False).head(25)

In [ ]:
FormattedData[FormattedData["total_count"]>10000].sort_values("success_rate",ascending=False)

In [ ]:
# average success rate is less
FormattedData["success_rate"].mean()

### summary
*  High total customers and succes rate place code is 28
*  low success rate and high customers count, places(50,15,30,8,9) 
* Region code affects the customer interest but average success rate of all region is less.

# Feature Engineering and Modeling

In [ ]:
train = rawdf
trainY = rawdf["Response"]

In [ ]:
train.info()

In [ ]:
import pandas as pd
num_feature = ["Age","Vintage","Annual_Premium"]
cat_feature = ["Gender","Driving_License","Region_Code","Previously_Insured","Vehicle_Age","Vehicle_Damage","Policy_Sales_Channel"]
train["Age_Cat"]= pd.cut(train["Age"],bins=[10,20,30,40,50,60,70,80,90,100],labels=[1,2,3,4,5,6,7,8,9])
train["Policy_Sales_Channel"] = train["Policy_Sales_Channel"].astype("int")
train["Region_Code"] = train["Region_Code"].astype("int")

ohe = OneHotEncoder(sparse=False)        
transformed_train_data = ohe.fit_transform(train[["Age_Cat","Gender","Vehicle_Age","Previously_Insured","Driving_License","Vehicle_Damage"]])

# # the above transformed_data is an array so convert it to dataframe
encoded_train_data = pd.DataFrame(transformed_train_data, index=train.index)        
train_data = pd.concat([train, encoded_train_data], axis=1)

In [ ]:
trainX = train_data.drop(["id","Age","Age_Cat","Gender","Vehicle_Age","Driving_License","Previously_Insured","Vehicle_Damage","Response"], axis=1)
trainX.columns

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score,confusion_matrix
logreg = LogisticRegression(random_state=0)
models = {
    "Logistic Regression":logreg,
#     "RandomForestClassifier":rf
}
features = ["Gender","Driving_License","Previously_Insured","Vehicle_Age","Vehicle_Damage"]
skf = StratifiedKFold(n_splits=5, shuffle=True)
for model_name,model in models.items():
    scores = []
    train_scores = []
    for train_indx,val_indx in skf.split(X=trainX,y=trainY):
        #get train and test data     
        X_train,X_val = trainX.loc[train_indx],trainX.loc[val_indx]
        Y_train,Y_val = trainY[train_indx],trainY[val_indx]
        sc = StandardScaler()
        train_sc = sc.fit_transform(X_train)
        val_sc  = sc.fit_transform(X_val)
        #train a model
        model.fit(train_sc,Y_train)
        #make a prediction
        Y_predict = model.predict_proba(val_sc)
        accuracy = roc_auc_score(Y_val,Y_predict[:,1])
        Ytrain_predict = model.predict_proba(train_sc)
        train_accuracy = roc_auc_score(Y_train,Ytrain_predict[:,1])        
        print("train",train_accuracy)
        print("test",accuracy)
        scores.append(accuracy)
        train_scores.append(train_accuracy)
    print("Mean Accurracy of test {0} is {1}".format(model_name,np.mean(scores)))
    print("Mean Accurracy of train {0} is {1}".format(model_name,np.mean(train_scores)))
    

In [ ]:

test["Age_Cat"]= pd.cut(test["Age"],bins=[10,20,30,40,50,60,70,80,90,100],labels=[1,2,3,4,5,6,7,8,9])
test["Policy_Sales_Channel"] = test["Policy_Sales_Channel"].astype("int")
test["Region_Code"] = test["Region_Code"].astype("int")

ohe = OneHotEncoder(sparse=False)        
transformed_test_data = ohe.fit_transform(test[["Age_Cat","Gender","Vehicle_Age","Previously_Insured","Driving_License","Vehicle_Damage"]])

# # the above transformed_data is an array so convert it to dataframe
encoded_test_data = pd.DataFrame(transformed_test_data, index=test.index)        
test_data = pd.concat([test, encoded_test_data], axis=1)
testX = test_data.drop(["id","Age","Age_Cat","Gender","Vehicle_Age","Previously_Insured","Driving_License","Vehicle_Damage"],axis=1)
sc = StandardScaler()
testt = sc.fit_transform(testX)
#predict
predictions = logreg.predict(testt)
output = pd.DataFrame({'id': test.id, 'Response': predictions})
print(output["Response"].value_counts())
output.to_csv('submission.csv', index=False)